In [134]:
import sys
import os
import time
import jpype
import jpype.imports
import json

In [112]:
try:
    sdk_gradle_home = os.environ["ATALA_PRISM_JARS"]
except KeyError:
    print("ERROR: `ATALA_PRISM_JARS` variable is not set.")
    print("Please, set it to the directory with Atala PRISM SDK dependencies JARs.")
    sys.exit(1)

In [113]:
try:
    jpype.imports.registerDomain('sdk', alias='io')
except ImportError as err:
    print(err.msg)
    sys.exit(ERROR)
jpype.startJVM(
    classpath=[
        os.path.join(sdk_gradle_home, f) for f in os.listdir(sdk_gradle_home)
            if f.endswith('.jar')
    ]
)

OSError: JVM is already started

In [114]:
from sdk.iohk.atala.prism.protos import *
from sdk.iohk.atala.prism.api.node import *
from sdk.iohk.atala.prism.api.models import *
from sdk.iohk.atala.prism.api import *
from sdk.iohk.atala.prism.crypto.derivation import *
from sdk.iohk.atala.prism.crypto.keys import *
from sdk.iohk.atala.prism.identity import *
from kotlinx.serialization.json import *
from kotlinx.serialization import DeserializationStrategy

KeyGenerator = KeyGenerator.INSTANCE
KeyDerivation = KeyDerivation.INSTANCE
MasterKeyUsage = MasterKeyUsage.INSTANCE
IssuingKeyUsage = IssuingKeyUsage.INSTANCE
RevocationKeyUsage = RevocationKeyUsage.INSTANCE
AuthenticationKeyUsage = AuthenticationKeyUsage.INSTANCE
AtalaOperationStatus = AtalaOperationStatus.INSTANCE
PrismDID = PrismDid

In [115]:
def wait_until_confirmed(node_api: NodePublicApi, operation_id: AtalaOperationId):
    """Waits until operation is confirmed by Cardano network
    
    Confirmation doesn't necessarily mean that operation was applied.
    For example, it could be rejected because of an incorrect signature or other reasons.

    :param node_api: Atala PRISM Node API object
    :type node_api: NodePublicApi
    :param operation_id: Atala PRISM operation ID
    :type operation_id: AtalaOperationId
    """
    operation_status = int(node_api.getOperationInfo(operation_id).join().getStatus())
    while operation_status != AtalaOperationStatus.getCONFIRMED_AND_APPLIED() \
        and operation_status != AtalaOperationStatus.getCONFIRMED_AND_REJECTED():
        print(f"Current operation status: {AtalaOperationStatus.asString(operation_status)}")
        time.sleep(1)
        operation_status = int(node_api.getOperationInfo(operation_id).join().getStatus())

def prepare_keys_from_mnemonic(mnemonic: MnemonicCode, password: str):
    """Creates a map of potentially useful keys out of a mnemonic code

    :param mnemonic: Mnemonic to generate keys from
    :type mnemonic: MnemonicCode
    :param password: Password for seed generation
    :type password: str
    :return: Map of keys
    :rtype: dict<String, ECKeyPair>
    """
    did_key_map = {}
    seed = KeyDerivation.binarySeed(mnemonic, password)
    did_key_map[PrismDid.getDEFAULT_MASTER_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, MasterKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_ISSUING_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, IssuingKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_REVOCATION_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, RevocationKeyUsage, 0)
    return did_key_map

def prepare_keys_from_seed(seed: bytes):
    """Creates a map of potentially useful keys out of a mnemonic code

    :param mnemonic: Mnemonic to generate keys from
    :type mnemonic: MnemonicCode
    :param password: Password for seed generation
    :type password: str
    :return: Map of keys
    :rtype: dict<String, ECKeyPair>
    """
    did_key_map = {}
    did_key_map[PrismDid.getDEFAULT_MASTER_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, MasterKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_ISSUING_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, IssuingKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_REVOCATION_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, RevocationKeyUsage, 0)
    return did_key_map

In [116]:
environment = "ppp-node-test.atalaprism.io"
node_auth_api = NodeAuthApiAsyncImpl(GrpcOptions("http", environment, 50053))

## Generating a DID

In [124]:
print(PrismDid.getDEFAULT_MASTER_KEY_ID())
print("Issuer: Generates and registers a DID")
issuer_keys = prepare_keys_from_mnemonic(KeyDerivation.randomMnemonicCode(), "passphrase")
issuer_unpublished_did = PrismDid.buildLongFormFromMasterPublicKey(
    issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPublicKey()
)
issuer_did = issuer_unpublished_did.asCanonical()
print(issuer_did.toString())
node_payload_generator = NodePayloadGenerator(
    issuer_unpublished_did,
    {
        PrismDid.getDEFAULT_MASTER_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey(),
        PrismDid.getDEFAULT_ISSUING_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_ISSUING_KEY_ID()].getPrivateKey()
    },
    
)
stored_object = {
    "did": issuer_unpublished_did.toString(),
    PrismDid.getDEFAULT_MASTER_KEY_ID(): str(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded()),
    PrismDid.getDEFAULT_ISSUING_KEY_ID(): str(issuer_keys[PrismDid.getDEFAULT_ISSUING_KEY_ID()].getPrivateKey().getHexEncoded())
}

print(issuer_unpublished_did)
print(PrismDid.getDEFAULT_MASTER_KEY_ID())
print(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey())
issuer_create_did_info = node_payload_generator.createDid(PrismDid.getDEFAULT_MASTER_KEY_ID())
issuer_create_did_operation_id = node_auth_api.createDid(
    issuer_create_did_info.getPayload(),
    issuer_unpublished_did,
    PrismDid.getDEFAULT_MASTER_KEY_ID()
).join()

print(f"""
- Issuer sent a request to create a new DID to PRISM Node.
- The transaction can take up to 10 minutes to be confirmed by the Cardano network.
- Operation identifier: {issuer_create_did_operation_id.hexValue()}
""")
create_did_operation_result = wait_until_confirmed(node_auth_api, issuer_create_did_operation_id)

print(f"- DID with id {issuer_did} is created")

master0
Issuer: Generates and registers a DID
did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0
did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0:Cj8KPRI7CgdtYXN0ZXIwEAFKLgoJc2VjcDI1NmsxEiEC6EdauexQCeIX0NE1MzW2az3ptoODGoUfoQNpVtxcs1Y
master0
io.iohk.atala.prism.crypto.keys.ECPrivateKey@2474df51


[main] INFO org.bitcoinj.crypto.MnemonicCode - PBKDF2 took 21.74 ms



- Issuer sent a request to create a new DID to PRISM Node.
- The transaction can take up to 10 minutes to be confirmed by the Cardano network.
- Operation identifier: a7050848f3cbe971a4cbc626c514bf376da9ef3e2a3d0fad473ff1db6619cd7f

Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
- DID with id did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0 is created


In [118]:
print(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded())
print(type(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey()))
bint = int(str(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded()),16)
kkee = ECPrivateKey(bint)

8acfa50c7df2a4077837fda719427df724c5c2c87a2e16fb16ba66f12eb6b027
<java class 'io.iohk.atala.prism.crypto.keys.ECPrivateKey'>


TypeError: No matching overloads found for constructor io.iohk.atala.prism.crypto.keys.ECPrivateKey(int), options are:
	public io.iohk.atala.prism.crypto.keys.ECPrivateKey(java.security.PrivateKey)



In [66]:
print(issuer_keys)
print(type(issuer_keys))

{'master0': <java object 'io.iohk.atala.prism.crypto.keys.ECKeyPair'>, 'issuing0': <java object 'io.iohk.atala.prism.crypto.keys.ECKeyPair'>, 'revocation0': <java object 'io.iohk.atala.prism.crypto.keys.ECKeyPair'>}
<class 'dict'>


In [125]:
# Issuer generates a credential to Holder identified by its unpublished DID
holder_did = PrismDid.fromString("did:prism:478c2a9e906b812531b606624ebd9e5a263ea62be5cdef78d69165976b736bad")
credential_claim = CredentialClaim(
    holder_did,
    JsonObject(
        {
            "name": JsonLiteral("Jose Lopez Portillo", True),
            "certificate": JsonLiteral("Certificate of PRISM SDK Python tutorial completion", True)
        }
    )
)
# node_payload_generator = NodePayloadGenerator(
#     issuer_unpublished_did,
#     {
#         PrismDid.getDEFAULT_MASTER_KEY_ID(): stored_object[PrismDid.getDEFAULT_MASTER_KEY_ID()],
#         PrismDid.getDEFAULT_ISSUING_KEY_ID(): stored_object[PrismDid.getDEFAULT_ISSUING_KEY_ID()],
#     },
    
# )


issue_credential_info = node_payload_generator.issueCredentials(
    PrismDid.getDEFAULT_ISSUING_KEY_ID(),
    [credential_claim]
)

In [127]:
#issuer_did2 = PrismDid.fromString(stored_object["did"])
issue_credential_batch_operation_id = node_auth_api.issueCredentials(
    issue_credential_info.getPayload(),
    issuer_unpublished_did.asCanonical(),
    PrismDid.getDEFAULT_ISSUING_KEY_ID(),
    issue_credential_info.getMerkleRoot()
).join()

issue_credential_batch_operation_result = \
    wait_until_confirmed(node_auth_api, issue_credential_batch_operation_id)

Current operation status: PENDING_SUBMISSION


In [130]:
holder_signed_credential = issue_credential_info.getCredentialsAndProofs()[0].getSignedCredential()
holder_credential_merkle_proof = issue_credential_info.getCredentialsAndProofs()[0].getInclusionProof()

print(f"""
Issuer [{issuer_did}] issued new credentials for the holder [{holder_did.asCanonical()}].
- issueCredentialBatch operation identifier: {issue_credential_batch_operation_id.hexValue()}
- Credential content: {holder_signed_credential.getContent()}
- Signed credential: {holder_signed_credential.getCanonicalForm()}
- Inclusion proof (encoded): {holder_credential_merkle_proof.encode()}
- Batch id: {issue_credential_info.getBatchId()}"""
)


Issuer [did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0] issued new credentials for the holder [did:prism:478c2a9e906b812531b606624ebd9e5a263ea62be5cdef78d69165976b736bad].
- issueCredentialBatch operation identifier: 4835ca1123ce314e8b1f38d7c991feecc5ed358b901fd5bf8e58799541f34b83
- Credential content: CredentialContent(fields={"id":"did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0","keyId":"issuing0","credentialSubject":{"name":"Jose Lopez Portillo","certificate":"Certificate of PRISM SDK Python tutorial completion","id":"did:prism:478c2a9e906b812531b606624ebd9e5a263ea62be5cdef78d69165976b736bad"}})
- Signed credential: eyJpZCI6ImRpZDpwcmlzbTpjNTdkZDcwMDc0YTFjYWM0NzQxZTYzMjA5MjQ4Mjg4ZTFmYjlmZmNkODg4N2JmNzMxZTBhZTc2YzQyYjA3OGIwIiwia2V5SWQiOiJpc3N1aW5nMCIsImNyZWRlbnRpYWxTdWJqZWN0Ijp7Im5hbWUiOiJKb3NlIExvcGV6IFBvcnRpbGxvIiwiY2VydGlmaWNhdGUiOiJDZXJ0aWZpY2F0ZSBvZiBQUklTTSBTREsgUHl0aG9uIHR1dG9yaWFsIGNvbXBsZXRpb24iLCJpZCI6ImRpZDpwcmlzbTo0

In [158]:
print(str(issue_credential_info.getBatchId().getId()))

67f0609570266a51305a731c4198ff07f89e7e1d13b0711952cf0d45e42d274f


In [29]:
# Verifier, who owns credentialClam, can easily verify the validity of the credentials
credential_verification_result = node_auth_api.verify(
    holder_signed_credential,
    holder_credential_merkle_proof
).join()

verification_errors = credential_verification_result.getVerificationErrors()
assert list(verification_errors) == [], "VerificationErrors should be empty"

AssertionError: VerificationErrors should be empty

In [83]:
mnemonic = KeyDerivation.randomMnemonicCode()
phrase = "password"
seed = bytes(KeyDerivation.binarySeed(mnemonic, phrase))

[main] INFO org.bitcoinj.crypto.MnemonicCode - PBKDF2 took 21.20 ms


In [84]:
print(type(seed))

<class 'bytes'>


In [85]:
keys1 = prepare_keys_from_mnemonic(mnemonic, phrase)

[main] INFO org.bitcoinj.crypto.MnemonicCode - PBKDF2 took 22.36 ms


In [86]:
keys2 = prepare_keys_from_seed(seed)

In [87]:
print(keys1[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded())

2447d8ab16457177306821ac64a7c49eb2a1b2f3d472b97dd42bcc3d7fd8481f


In [88]:
print(keys2[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded())

2447d8ab16457177306821ac64a7c49eb2a1b2f3d472b97dd42bcc3d7fd8481f


In [107]:
cl = JsonObject(
        {
            "name": JsonLiteral("Jose Lopez Portillo", True),
            "certificate": JsonLiteral("Certificate of PRISM SDK Python tutorial completion", True)
        }
    )
jsonString = '{"firstName":"Tom", "lastName": "Broody"}'

In [108]:
print(type(cl))

<java class 'kotlinx.serialization.json.JsonObject'>


In [42]:
pp = Json.decodeFromString(DeserializationStrategy.deserialize(),jpype.java.lang.String('{"casa": "asdsa"}'))

TypeError: No matching overloads found for *static* kotlinx.serialization.DeserializationStrategy.deserialize(), options are:
	public abstract java.lang.Object kotlinx.serialization.DeserializationStrategy.deserialize(kotlinx.serialization.encoding.Decoder)



In [43]:
pp2 = Json.decodeFromDynamic('{"casa": "asdsa"}')

AttributeError: type object 'kotlinx.serialization.json.Json' has no attribute 'decodeFromDynamic'

In [88]:
mijson = {
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1"
  ],
  "id": "http://example.edu/credentials/3732",
  "type": ["VerifiableCredential", "UniversityDegreeCredential"],
  "issuer": "https://example.edu/issuers/14",
  "issuanceDate": "2010-01-01T19:23:24Z",
  "issuanceDate2": 2,
  "issuanceDate2": 22.2,
  "credentialSubject": {
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "degree": {
      "type": "BachelorDegree",
      "name": "Bachelor of Science and Arts",
      "score": 2  
    }
  },
  "credentialStatus": {
    "id": "https://example.edu/status/24",
    "type": "CredentialStatusList2017"
  }
}

In [85]:
for key in mijson:
    print(key, '->', type(mijson[key]))

context -> <class 'list'>
id -> <class 'str'>
type -> <class 'list'>
issuer -> <class 'str'>
issuanceDate -> <class 'str'>
issuanceDate2 -> <class 'float'>
credentialSubject -> <class 'dict'>
credentialStatus -> <class 'dict'>


In [108]:
def dictToJsonObjt(json: dict):
    resp = {}
    for key in json:
        if isinstance(json[key], str) or isinstance(json[key], int) or isinstance(json[key], float): 
            resp[key] = JsonLiteral(json[key], True)
        elif isinstance(json[key], dict): 
            resp[key] = dictToJsonObjt(json[key])
        elif isinstance(json[key], list): 
            resp[key] = listToJsonArray(json[key])
    return JsonObject(resp)

def listToJsonArray(array: list):
    resp = jpype.java.util.ArrayList()
    for el in array:
        if isinstance(el, str) or isinstance(el, int) or isinstance(el, float): resp.add(JsonLiteral(el, True))
        elif isinstance(el, dict): 
            resp.add(dictToJsonObjt(el))
        elif isinstance(el, dict): 
            resp.add(listToJsonArray(el))
    return JsonArray(resp)

In [109]:
f=dictToJsonObjt(mijson)

In [97]:
print(f)

java.lang.ClassCastException: java.lang.ClassCastException: class kotlinx.serialization.json.JsonArrayBuilder cannot be cast to class kotlinx.serialization.json.JsonElement (kotlinx.serialization.json.JsonArrayBuilder and kotlinx.serialization.json.JsonElement are in unnamed module of loader 'app')

In [105]:
gg = JsonArray(["asd"])

TypeError: No matching overloads found for constructor kotlinx.serialization.json.JsonArray(list), options are:
	public kotlinx.serialization.json.JsonArray(java.util.List)



In [99]:
gg.add(JsonLiteral("asd", True))

True

In [107]:
pp = jpype.java.util.List

TypeError: Cannot create Java interface instances


In [93]:
llsd={}

In [94]:
llsd["asdad"] =4